# Question I Choose
## Predict the percentage of residential customers (RES.CUST.PCT) affected by a major power outage.

**Name(s)**: Luke Lin, Andrew Yin

**Website Link**: (your website link)

## Code

In [2]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Framing the Problem

In [9]:
# Read the Excel file into a pandas DataFrame
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")

# Drop informational rows
outage_cleaned = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names based on the first row
outage_cleaned.columns = outage_cleaned.iloc[0]

# Drop rows related to units and variables
outage_cleaned = outage_cleaned.drop([4, 5])
outage_cleaned = outage_cleaned.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a new datetime column
outage_cleaned['OUTAGE.START'] = pd.to_datetime(outage_cleaned['OUTAGE.START.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a new datetime column
outage_cleaned['OUTAGE.RESTORATION'] = pd.to_datetime(outage_cleaned['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Replace "NA" with NaN for missing values
outage_cleaned.replace("NA", np.nan, inplace=True)

#Essential Columns Analysis
essential_col = [
    "U.S._STATE", 
    'OUTAGE.START',
    "OUTAGE.RESTORATION", 
    "CUSTOMERS.AFFECTED", 
    "OUTAGE.DURATION", 
    "DEMAND.LOSS.MW", 
    "RES.PRICE", 
    "NERC.REGION", 
    "CLIMATE.REGION",
    "CAUSE.CATEGORY",
    "TOTAL.SALES"
]
# Display the cleaned DataFrame
# outage_cleaned = outage_cleaned[essential_col]
print(outage_cleaned.shape)
outage_cleaned.head().to_clipboard()

(1534, 54)


## More Cleaning and Encoding
(add details later)

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

processed_outage = outage_cleaned.copy()

# Fill numerical columns with the mean
num_cols = processed_outage.select_dtypes(include=[np.number]).columns
processed_outage[num_cols] = processed_outage[num_cols].fillna(processed_outage[num_cols].mean())

# Fill categorical columns with the mode
cat_cols = processed_outage.select_dtypes(include=['object']).columns
processed_outage[cat_cols] = processed_outage[cat_cols].fillna(processed_outage[cat_cols].mode().iloc[0])

# 2. Convert Date/Time Columns
processed_outage['OUTAGE.START'] = pd.to_datetime(processed_outage['OUTAGE.START'])
processed_outage['OUTAGE.RESTORATION'] = pd.to_datetime(processed_outage['OUTAGE.RESTORATION'])

# Create a new feature for outage duration in minutes
processed_outage['OUTAGE.DURATION'] = (processed_outage['OUTAGE.RESTORATION'] - processed_outage['OUTAGE.START']).dt.total_seconds() / 60

# 3. Encode Categorical Variables
le = LabelEncoder()
for col in cat_cols:
    processed_outage[col] = le.fit_transform(processed_outage[col])


### Baseline Model

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Select features and target
features = processed_outage[['U.S._STATE', 'OUTAGE.DURATION']]
target = processed_outage['RES.CUST.PCT']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Define preprocessing for numerical columns (scale them)
num_processor = Pipeline([
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical columns (encode them)
cat_processor = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_processor, ['OUTAGE.DURATION']),
        ('cat', cat_processor, ['U.S._STATE'])
    ])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

Test RMSE: 21.38164939292652


## Improve Baseline

In [19]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'regressor__fit_intercept': [True, False]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit it to the data and find the best parameters
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding RMSE on the training set
print('Best parameters:', grid_search.best_params_)
print('Best RMSE on training set:', np.sqrt(-grid_search.best_score_))

# Evaluate the best model on the test set
y_pred = grid_search.predict(X_test)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

Best parameters: {'regressor__fit_intercept': False}
Best RMSE on training set: 23.26830298012474
Test RMSE: 21.381654948380188


### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO